In [ ]:
import re
from copy import deepcopy
from functools import partial
from operator import add, mul

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dustgoggles.composition import Composition
from fs.osfs import OSFS
from scipy.ndimage import gaussian_filter

from marslab.compat.clementine import MOSAIC_SPECIAL_CONSTANTS
from marslab.bandset.clementine import ClemBandSet
import marslab.extras.cmaps
from marslab.demos.config import gen_spectop_defaults
from marslab.bandset.mastcam import (
    bulk_scrape_mcam_metadata, parse_mcam_files, McamBandSet
)
from marslab.bandset import ImageBands
from marslab.imgops.imgutils import std_clip, normalize_range
from marslab.imgops.look import Look
from marslab.imgops.render import colormapped_plot, simple_figure

rng = np.random.default_rng()
c = partial(plt.close, 'all')

mpl.rcParams['image.cmap'] = 'Greys_r'
mpl.rcParams['figure.figsize'] = [15, 15]

%matplotlib qt

In [ ]:
# demo data is stashed here:
# https://drive.google.com/drive/folders/1dDkgof792KGqnO3g_qkZJ5aZXJc0MHwI

# define which files we're grabbing
mcam_data = OSFS('/home/michael/Desktop/mcam_data')
seq_id = 13201
caltype = 'DRXX'
search = partial(re.search, f'{seq_id}.*{caltype}.*LBL')
results = tuple(
    map(mcam_data.getsyspath, filter(search, mcam_data.listdir('')))
)
results 

In [ ]:
c()

In [ ]:
info = pd.concat([
    # little text-and-filename parsing functions
    pd.DataFrame(parse_mcam_files(results)),
    pd.DataFrame(bulk_scrape_mcam_metadata(results))
], axis=1)
info["PATH"] = results
info["PATH"] = info["PATH"].str.replace("LBL", "IMG")
info = info.sort_values(by='CSEQ').reset_index(drop=True)
# this sequence begins with a full spin of the right-eye filters before
# the stereo images we're interested in, so we cut command sequence < 8.
info = info.loc[info['CSEQ'] >= 8].reset_index(drop=True)
info

In [ ]:
# construct the marslab.imgops.bandset.BandSet object.
# some specific behaviors are defined on McamBandSet, 
# which references some contents from marslab.compat.xcam 

observation = McamBandSet(info)
# 'metadata' is a DataFrame. important columns include:
# BAND -- human-readable name of the band
# IX -- index of the band within a file 
# (no value implies single-band files or a mistake)
observation.metadata

In [ ]:
# pdr_load() leverages pdr to get info from PDS labels. 
# applies scale and offset found in labels by default.
observation.load_method

In [ ]:
observation.load('all')

# raw images are a dict of str: ndarray
observation.raw.keys()

In [ ]:
# some filters fall within the bandpasses of the camera's bayer array
plt.imshow(observation.raw['L2'])

In [ ]:
observation.metadata[['BAND', 'BAYER_PIXEL']]

In [ ]:
observation.bayer_info

In [ ]:
# debayer all images and place in observation.debayered
observation.bulk_debayer('all')
# pick 'correct' version of image
plt.imshow(observation.get_band('L2'))

In [ ]:
# image types commonly used in MCAM publications: 
# parameter map of band depth at 527 nm, 'natural color' through the clear filter,
# and a decorrelation stretch  (simple dimensionality reduction)

bd527 = {'look': 'band_depth', 'bands': ('L2', 'L4', 'L1'), 'name':'bd527'}
dcs = {
    'name': 'exciting dcs',
    'look': 'dcs', 
    'bands': ('L3', 'L1', 'L2'), 
    'params': {'contrast_stretch': 0.9, 'sigma': None}
}
natural = {
    'name': 'natural color',
    'look': 'composite', 
    'prefilter': {'function': normalize_range}, 
    'bands': ('L0R', 'L0G', 'L0B')
}

In [ ]:
observation.make_look_set([bd527, natural, dcs])
# looks, like raw images, are cached in a dict -- the bandset's 'looks' attribute.
observation.looks.keys()

In [ ]:
# display these real quick
def shew(things, **show_kwargs):
    return [plt.figure().add_subplot().imshow(thing, **show_kwargs) for thing in things]

shew(observation.looks.values())

In [ ]:
# mostly-shared pipeline for spectral parameter maps
fancy_spect_defaults = gen_spectop_defaults(special_constants=[0])
fancy_spect_defaults

In [ ]:
# individual parameter settings

# 'look' is the name of a spectral parameter function from marslab.spectops, 
# a rendering function from marslab.imgops.render, or any callable with a 
# compatible signature
mcam_spectop_looks = [
    {'look': 'band_depth', 'bands': ('L2', 'L4', 'L1'), 'name':'bd527'},
    {'look': 'band_depth', 'bands': ('R3', 'R6', 'R5'), 'name': 'bd827'},
    {'look': 'ratio', 'bands': ('R6', 'R2'), 'name': 'r62'}
]
# inserting template here
for look in mcam_spectop_looks:
    look |= fancy_spect_defaults
    
# add differently-colored versions

grays = [deepcopy(look) for look in mcam_spectop_looks]
rainbows = [deepcopy(look) for look in mcam_spectop_looks]
for colormap, colorlooks in zip(('Greys_r', 'jet_r'), (grays, rainbows)): 
    for colorlook in colorlooks:
        colorlook['plotter']['params']['cmap'] = colormap
        colorlook['name'] = colorlook['name'] + f' {colormap}'

# create matplotlib figures from dcs
dcs['plotter'] = {'function': simple_figure}
# make an alternative dcs
dcs_2 = deepcopy(dcs)
dcs_2['params'] = {'sigma': 0.9, 'contrast_stretch': 0.3, 'special_constants': [0]}
dcs_2['name'] = 'dcs_boring'
stretchy = [dcs, dcs_2]

# and similarly with natural and 'enhanced' color images
natural['plotter'] = {'function': simple_figure}
enhanced = deepcopy(natural)
enhanced["name"] = "enhanced color l3l1l2"
enhanced['bands'] = ("L3", "L1", "L2")
enhanced['prefilter']['params'] = {"stretch": (1.25, 1)}

looks = mcam_spectop_looks + grays + rainbows + stretchy + [enhanced, natural]

In [ ]:
[look['name'] for look in looks]

In [ ]:
looks[3]

In [ ]:
observation.purge('looks')

In [ ]:
# this supports multithreading, but it doesn't work well in a REPL environment,
# so I'm not demonstrating it here.
# observation.threads = {'look': 8}
observation.make_look_set(looks)

In [ ]:
observation.looks.keys()

In [ ]:
c()

In [ ]:
tycho_uvvis = ClemBandSet('demos/data/clementine/uvvis_52s_005e.tif')
tycho_uvvis.load('all')

In [ ]:
tycho_uvvis.metadata

In [ ]:
plt.imshow(np.clip(tycho_uvvis.get_band('C'), 0, 65536))

In [ ]:
CLEM_SPECTOP_DEFAULTS = {
    'params': {'special_constants': MOSAIC_SPECIAL_CONSTANTS},
    "limiter": {"function": std_clip, "params":{'sigma': 2}},
    "postfilter": {"function": gaussian_filter, "params": {"sigma": 1}},
    "plotter": {
        "function": colormapped_plot,
        "params": {
            "cmap": "orange_teal",
            "render_colorbar": True,
            "special_constants": MOSAIC_SPECIAL_CONSTANTS
        },
    },
}

In [ ]:
clem_spectops = (
    {'name': 'rea', 'bands': ('E', 'A'), 'look': 'ratio'},
    {'name': 'rcb', 'bands': ('C', 'B'), 'look': 'ratio'},
)
clem_stretches = [
    {
        "name": "dcs bde",
        "look": "dcs",
        "params": {
            "special_constants": MOSAIC_SPECIAL_CONSTANTS, 
            "contrast_stretch": 1, 
            "sigma": None
        },
        "plotter": {"function": simple_figure},
        "bands": ("B", "D", "E")
}
]
fancy_spect_defaults = gen_spectop_defaults(special_constants=[0])

clem_looks = [
     gen_spectop_defaults(special_constants=MOSAIC_SPECIAL_CONSTANTS)
    | look for look in clem_spectops
] + clem_stretches

In [ ]:
c()

In [ ]:
tycho_uvvis.make_look_set(clem_looks)

In [ ]:
tycho_uvvis.save_looks('.')

In [ ]:
# ImageBands is intended for simple multiband operations on consumer
# image formats. It uses pillow and takes the channel names of the colorspace as
# the names of the spectral bands.
eclipse = ImageBands('demos/data/pictures/Lunar_eclipse_al-Biruni.jpg')

# add nominal wavelengths -- common band centers for consumer cameras
consumer_bayer_waves = {'R': 596, 'G': 524, 'B': 458}
eclipse.metadata['WAVELENGTH'] = consumer_bayer_waves.values()

eclipse.metadata

In [ ]:
eclipse.load_method

In [ ]:
shew(list(eclipse.raw.values()), cmap='Greys_r')

In [ ]:
c()

In [ ]:
ratio_rg_instruction = {
    "look": "ratio",
    "special_constants": [0],
    # bands from the 
    "bands": ("R", "G"),
    "limiter": {"function": std_clip, "params":{'sigma': 1}},
    # a plotting function from marslab.imgops.render, or callable with a 
    #  compatible signature
    "plotter": {"function": colormapped_plot, "params": {"cmap": "orange_teal"}}
}

In [ ]:
ratio_rg = Look.compile_from_instruction(ratio_rg_instruction)
ratio_rg

In [ ]:
ratio_rg_plot = ratio_rg.execute(
    [eclipse.raw[band] for band in ratio_rg_instruction['bands']],
)

In [ ]:
# we can also cache every step of these pipelines.
ratio_rg.add_capture('limiter')
ratio_rg.execute(
    [eclipse.raw[band] for band in ratio_rg_instruction['bands']],
)


In [ ]:
ratio_rg.captures['limiter']


In [ ]:
plt.imshow(ratio_rg.captures['limiter'])

In [ ]:
# or place callbacks in them.
ratio_rg = Look.compile_from_instruction(ratio_rg_instruction)
ratio_rg.add_send(plt.imshow, 'limiter')
# ratio_rg.execute(
#     [eclipse.raw[band] for band in ratio_rg_instruction['bands']],
# )

In [ ]:
# they can also be lazily evaluated.
ratio_rg = Look.compile_from_instruction(ratio_rg_instruction)
ratio_rg_stepper = ratio_rg.itercall(
    [
        eclipse.raw[band] for band in ratio_rg_instruction['bands']
    ]
)

In [ ]:
ratio_rg_stepper

In [ ]:
next(ratio_rg_stepper)

In [ ]:
x, y = np.meshgrid(np.arange(128), np.arange(128))
weight = np.sqrt((64 - x) ** 2 + (32 - y) ** 2)
normal = rng.normal(128, 30, (128, 128)) * (90 - weight)
plt.imshow(normal)

In [ ]:
normal

In [ ]:
tile_display = Composition({
    'tile': np.tile,
    'mirror_horizontal': np.fliplr,
    'plot': plt.imshow
})

In [ ]:
# this won't work: np.tile lacks a required positional argument 
tile_display.execute(normal)

In [ ]:
# however, we can bind arguments to these objects after initialization
comps = {n: deepcopy(tile_display) for n in range(1, 11)}
for comp_ix, comp in comps.items():
    if comp_ix % 2 == 0:
        comp.bind_kwargs('tile', reps=(comp_ix, 2))
    else:
        comp.bind_kwargs('tile', reps=(2, comp_ix))
    if comp_ix % 3 == 0:
        comp.bind_kwargs('plot', cmap='orange_teal')
    else:
        comp.bind_kwargs('plot', cmap='viridis')
        
for comp in comps.values():
    comp.add_trigger(plt.figure,'tile')
    comp.add_trigger(plt.colorbar, 'plot')
    
itercomps = iter(comps.values())

In [ ]:
next(itercomps).execute(normal)

In [ ]:
c()

In [ ]:
arithmetic = Composition([add, mul])
arithmetic.add_insert(None, 0)
arithmetic.add_insert(None, 1)
# args can be inserted at execution time -- this can also be used as a hack
# to partially apply arguments to functions with positional-only arguments
arithmetic.execute(1, 1, 2)

In [ ]:
def head(fn, size):
    with open(fn) as file:
        return file.read(size)
    
sed = Composition({
    'head': head,
    'replace': re.sub,
    'cat': print
})

sed.bind_args('head', size=100)
sed.bind_args(
    'replace', 
    '.*import.*', 
    'raise ValueError("dependencies are forbidden")',
)
sed.add_send(print, 'head')
sed.execute('tests/test_bandset.py')

In [ ]:
sed.bind_args(
    'replace', 
    '.*import.*', 
    'import marslab',
    flags=re.M+re.DOTALL,
    rebind=True
)
sed.execute('tests/test_bandset.py')